# NVD3 Visualizations

## Procedural vs Declarative Visualizations

**Procedural :**
- The actions which are to be taken are defined successively.
- Example : matplotlib.pyplot

**Declarative :**
- The models of the relationship are defined (declared) before the plotting starts.
- Example : D3.js

NVD3 is a Python wrapper for d3.js library which is used for declarative visualizations.

## Installing python-nvd3

In [1]:
!pip install python-nvd3

In [2]:
import nvd3
import pandas as pd
import operator
# Initializing for the notebook
nvd3.ipynb.initialize_javascript()

loaded nvd3 IPython extension
run nvd3.ipynb.initialize_javascript() to set up the notebook
help(nvd3.ipynb.initialize_javascript) for options


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Getting the Data

In [3]:
# import data
import pickle
data = pickle.load(open('data_list','rb'))

In [4]:
df = pd.DataFrame(data)
df = df.drop(["albumArtRef", "artistArtRef", "albumId", "artistId", "clientId", "nid", "storeId"], axis=1)

In [5]:
df["durationMillis"] = pd.to_numeric(df["durationMillis"])
df["estimatedSize"] = pd.to_numeric(df["estimatedSize"])

In [6]:
df.describe()

,discNumber,durationMillis,estimatedSize,playCount,trackNumber,year
count,736.000000,736.000000,7.360000e+02,732.000000,736.000000,735.000000
mean,1.013587,253180.706522,1.014937e+07,24.524590,5.010870,2008.209524
std,0.115847,73870.938806,2.954185e+06,15.408197,4.822272,11.308875
min,1.000000,30000.000000,1.202704e+06,1.000000,1.000000,1902.000000
25%,1.000000,211000.000000,8.459782e+06,13.000000,1.000000,2006.000000
50%,1.000000,238000.000000,9.532630e+06,21.000000,3.000000,2012.000000
75%,1.000000,284250.000000,1.140143e+07,32.000000,7.000000,2015.000000
max,2.000000,738000.000000,2.952571e+07,92.000000,49.000000,2018.000000


In [7]:
def get_artist_count(data):
    albumArtist = {}
    for each_song in data:
        if(albumArtist.get(each_song['albumArtist'])):
                albumArtist[each_song['albumArtist']]+=1
        else:
                albumArtist[each_song["albumArtist"]]=1
    
    album_artist_sorted = sorted(albumArtist.items(),key = operator.itemgetter(1),reverse=True)
    album_artist = [x[0] for x in album_artist_sorted]
    artist_count = [x[1] for x in album_artist_sorted]
    
    return album_artist,artist_count

In [42]:
def get_played_count(data):
    artist_count = {}
    for each_song in data:
        try:
            count = each_song['playCount']
        except:
            count = 0
        artist = each_song['albumArtist']
        
        if(artist_count.get(artist)):
            artist_count[artist]+=count
        else:
            artist_count[artist] = count
            
    album_artist_sorted = sorted(artist_count.items(),key = operator.itemgetter(1),reverse=True)
    album_artist = [x[0] for x in album_artist_sorted]
    artist_count = [x[1] for x in album_artist_sorted]
    
    return album_artist,artist_count

In [52]:
def top_ten_songs(data):
    song_count = {}
    
    for each_song in data:

        try:
            song_name = each_song['title']
        except:
            song_name = "NIL"
            
        try:
            song_count_curr = each_song['playCount']
        except:
            song_count_curr = 0
        
        if(song_count.get(song_name)):
            song_count[song_name]+=song_count_curr
        
        else:
            song_count[song_name]=song_count_curr
    
    song_count_sorted =  sorted(song_count.items(),key = operator.itemgetter(1),reverse=True)
    song = [x[0] for x in song_count_sorted]
    song_cnt = [x[1] for x in song_count_sorted]
    
    return song,song_cnt

In [64]:
def song_length(data):  
    duration_minute = []  
    for each_song in data:     
        duration_minute.append(int(each_song['durationMillis'])/(1000*60))     
    return duration_minute

## Bar Charts

In [65]:
def makebarchart(data,filename):
    xdata,ydata=data
    barchart = nvd3.discreteBarChart(name = 'discreteBarChart',height = 800, width = 1200)
    barchart.add_serie(y=ydata[:10],x=xdata[:10])
    barchart.buildhtml()
    chart_html = barchart.htmlcontent
    with open(filename +'.html','w') as f:
        f.write(chart_html)


In [66]:
makebarchart(get_artist_count(data),'albumartist')

In [67]:
makebarchart(get_played_count(data),'artistplayed')

In [68]:
makebarchart(top_ten_songs(data),'top10songs')


In [71]:
makebarchart((range(len(song_length(data))),song_length(data)),'songlength')